In [1]:
import os
from os.path import join as oj
import sys, time
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
import data_pecarn
import data_psrc
from data import classification_setup, get_feat_names

import matplotlib.gridspec as grd

# sns.set(style="black")
# plt.style.use('dark_background')
%matplotlib inline
%load_ext autoreload
%autoreload 2

# look at all dsets

In [31]:
data_file='data_psrc/psrc_data_processed.csv'
df = pd.read_csv(data_file)


NUM_PATIENTS = df.shape[0]
print(NUM_PATIENTS)

# fix col names
df['id'] = np.arange(1, NUM_PATIENTS + 1)
df = df.rename(columns=lambda x: x.replace('choice=0ne', 'choice=None').strip())
df = df.replace('0ne', 'None')
df['ageinyrs'] = df['Age in years'].fillna(0) + df['Age in months'].fillna(0) / 12

# drop unnecessary
df = df.drop(columns=['Record ID',  'Arrival time in ED', 'Age', 'Age in months', 'Age in years'])

# done
df = df.rename(columns={'Seatbelt sign': 'SeatBeltSign', 
                        'Initial GCS': 'GCSScore',
                        'Lower chest wall/costal margin tenderness to palpation  (choice=1 on left)': 'LtCostalTender',
                        'Lower chest wall/costal margin tenderness to palpation  (choice=1 on right)': 'RtCostalTender'
                       })

# fill with unknown
df['GCSScore'] = (df['GCSScore'].fillna(df['GCSScore'].median())).astype(int)
df = df.fillna('unknown')

2188


In [32]:
# these need matching
df = df.rename(columns={'Abdominal distension': 'AbdDistention',
                        'Abdominal tenderness to palpation': 'AbdTenderDegree',
                       })
binary = {
    0: 'no',
    1: 'yes',
    'unknown': 'unknown'
}
df['AbdDistention'] = [binary[v] for v in df.AbdDistention.values]

abdTenderDegree = {
    'None': 'Mild',
    'Mild': 'Mild',
    'Moderate': 'Moderate',
    'Severe': 'Severe',
    'Limited exam secondary to intubation/sedation': 'unknown',
    'unknown': 'unknown'
}
df['AbdTenderDegree'] = [abdTenderDegree[v] for v in df.AbdTenderDegree.values]

moi = {
    'Mechanism of injury (choice=Assault/struck)': 'Object struck abdomen',
    'Mechanism of injury (choice=ATV injury)': 'Motorcycle/ATV/Scooter collision',
    'Mechanism of injury (choice=Bike crash)': 'Bike collision/fall',
    'Mechanism of injury (choice=Bike struck by auto)': 'Pedestrian/bicyclist struck by moving vehicle',
    'Mechanism of injury (choice=Fall > 10 ft. height)': 'Fall from an elevation',
    'Mechanism of injury (choice=Golf cart injury)': 'Motorcycle/ATV/Scooter collision',
    'Mechanism of injury (choice=Motorcycle/dirt bike crash)': 'Motorcycle/ATV/Scooter collision',
    'Mechanism of injury (choice=MVC)': 'Motor vehicle collision',
    'Mechanism of injury (choice=Pedestrian struck by auto)': 'Pedestrian/bicyclist struck by moving vehicle',
    'Mechanism of injury (choice=Other blunt mechanism)': 'Object struck abdomen',
}
df['RecodedMOI'] = ['unknown'] * df.shape[0]

'''
for k in moi:
    print(k, (df[k]==1).sum())
    df.loc[df[k] == 1, 'RecodedMOI'] = moi[k]
print('tot', (df['RecodedMOI']!='unknown').sum())
'''
# other vars present
# MOI
# Race

# Thoracic injury - matches ThoracicTrauma


"\nfor k in moi:\n    print(k, (df[k]==1).sum())\n    df.loc[df[k] == 1, 'RecodedMOI'] = moi[k]\nprint('tot', (df['RecodedMOI']!='unknown').sum())\n"

In [33]:
list(df.keys())

['Race',
 'Hispanic ethnicity',
 'Mechanism of injury (choice=Assault/struck)',
 'Mechanism of injury (choice=ATV injury)',
 'Mechanism of injury (choice=Bike crash)',
 'Mechanism of injury (choice=Bike struck by auto)',
 'Mechanism of injury (choice=Fall > 10 ft. height)',
 'Mechanism of injury (choice=Golf cart injury)',
 'Mechanism of injury (choice=Motorcycle/dirt bike crash)',
 'Mechanism of injury (choice=MVC)',
 'Mechanism of injury (choice=Pedestrian struck by auto)',
 'Mechanism of injury (choice=Other blunt mechanism)',
 'If bike crash, landed on handle bar',
 'MVC details (choice=>40 MPH)',
 'MVC details (choice=Ejection)',
 'MVC details (choice=Rollover)',
 'MVC details (choice=Prolonged extrication)',
 'MVC details (choice=Unrestrained)',
 'MVC details (choice=Death in vehicle)',
 'MVC details (choice=0t reported)',
 'Other mechanism of injury',
 'Alert level',
 'Complainabd. pain',
 'Emesis post injury',
 'Intubated prior to arrival',
 'Intubated during trauma resuscitati

In [34]:
d = classification_setup(df)
# d = df
feat_names, common_feats = get_feat_names(d)
common_feats

['AbdDistention_no',
 'AbdDistention_unknown',
 'AbdDistention_yes',
 'AbdTenderDegree_Mild',
 'AbdTenderDegree_Moderate',
 'AbdTenderDegree_Severe',
 'AbdTenderDegree_unknown',
 'GCSScore',
 'LtCostalTender',
 'RecodedMOI_unknown',
 'RtCostalTender',
 'SeatBeltSign',
 'ageinyrs']

In [8]:
# outcomes
iai_keys = [k for k in df.keys() if 'Interventions for IAI' in k]
iai_with_intervention_keys = [k for k in iai_keys if not 'choice=None' in k]
outcomes = ['Admission', 
            'ICU admission', 
            'Length of inpatient stay (days)',
            'Delayed inpatient diagnosis of IAI (more than 24 hours after admission)',
            'Mortality (within 30 days of injury)',
            'Mortality related to trauma',
            'Mortality secondary to intra-abdominal injury',
            'Missed diagnosis of IAI (following discharge)'
            ] + iai_keys

# iai
df['iai'] = df[iai_keys].sum(axis=1) > 0
df['iai_with_intervention'] = df[iai_with_intervention_keys].sum(axis=1) > 0

### eda plots

In [4]:
fnames = df.keys()
missing_arr = np.zeros((len(fnames), data.NUM_PATIENTS + 1), dtype=np.bool)

for i, fname in enumerate(fnames):
#     df = r[fname]
    ids = np.unique(df.id.values)
#     print(np.min(ids), np.max(ids))
    missing_arr[i, ids] = 1

In [ ]:
plt.figure(figsize=(5, 10))
sns.barplot(x=missing_arr.sum(axis=1), y=fnames, orient='h')
plt.xlabel('number of patients')
plt.show()

In [ ]:
# plt.imshow(not_missing)
plt.figure(figsize=(12, 20), dpi=300)
sns.clustermap(missing_arr, yticklabels=fnames)
# plt.tight_layout()
plt.show()

In [ ]:
# Compute the correlation matrix
corr = df.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(10, 220, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
# sns.clustermap(corr) #, mask=mask, cmap=cmap, vmax=.3, center=0,
#             square=True, linewidths=.5, cbar_kws={"shrink": .5})

# look at demographics

In [19]:
# demographics = pd.read_csv('iaip_data/Datasets/demographics.csv').rename(columns={'SubjectID': 'id'})
# ids_np = np.array(list(ids)) - 1
ids_np = df.iai_intervention == 1
# vals = demographics.values[ids_np]
# print(list(demographics))
print(df.keys()[:10])

Index(['id', 'SEX', 'RACE', 'OTHER_RACE', 'HISPANIC_ETHNICITY', 'ageinyrs',
       'EDTriageDay_1', 'EDTriageTime_1', 'InjuryDay_1', 'InjuryTime_1'],
      dtype='object')


In [21]:
# the mean age is the only one which is wrong
print('age', np.mean(df.ageinyrs[ids_np]), np.mean(df.ageinyrs))
print('age<2', np.sum(df.ageinyrs[ids_np] < 2), np.sum(df.ageinyrs < 2))
print('sex', np.sum(df.SEX[ids_np] == 'M'), np.sum(df.SEX == 'M'))
print('hispanic ethnicity', np.sum(df.HISPANIC_ETHNICITY[ids_np] == 0), np.sum(df.HISPANIC_ETHNICITY == 0))

age 9.339901477832512 9.767270009963468
age<2 10 1167
sex 125 7384
hispanic ethnicity 119 7656
